In [ ]:
import pandas as pd

caminho_absoluto = r'C:\Users\b3rna\Documents\Projetos\analise_sentimentos\olist_order_reviews_dataset.csv'

# Agora, carregue o arquivo usando essa variável
df_reviews = pd.read_csv(caminho_absoluto)

# Exibindo as 5 primeiras linhas do dataframe para entender a estrutura
print("As 5 primeiras linhas do nosso dataset:")
print(df_reviews.head())

# Exibindo informações gerais sobre o dataset (tipos de dados, valores nulos, etc.)
print("\nInformações sobre o dataset:")
df_reviews.info()

# Verificando a distribuição das notas (review_score)
print("\nDistribuição das notas de review:")
print(df_reviews['review_score'].value_counts())

In [ ]:
# Removendo as linhas onde não há comentário de texto.
# O .dropna() remove linhas com valores nulos (NaN) na coluna especificada.
df_limpo = df_reviews.dropna(subset=['review_comment_message'])

# 2. Criando a Coluna "sentimento"
def classificar_sentimento(nota):
    if nota >= 4:
        return 'positivo'
    elif nota == 3:
        return 'neutro'
    else: # notas 1 e 2
        return 'negativo'

# Aplicando a função na coluna 'review_score' para criar a nova coluna 'sentimento'
# Usando .loc para evitar aquele aviso de "SettingWithCopyWarning"
df_limpo.loc[:, 'sentimento'] = df_limpo['review_score'].apply(classificar_sentimento)

# 3. Selecionando as Colunas Relevantes
# Agora, criamos um DataFrame final apenas com o texto e o sentimento.
df_final = df_limpo[['review_comment_message', 'sentimento']].copy()
df_final.rename(columns={'review_comment_message': 'texto'}, inplace=True)

# Exibindo as 5 primeiras linhas do nosso novo DataFrame para confirmar
print("DataFrame limpo e pronto para análise:")
print(df_final.head())

# Verificando a nova distribuição de sentimentos
print("\nNova distribuição de sentimentos:")
print(df_final['sentimento'].value_counts())

In [ ]:
import nltk
import string

# Baixando a lista de stopwords em português (só precisa rodar uma vez)
nltk.download('stopwords')

# Pegando a lista de stopwords
stopwords = nltk.corpus.stopwords.words('portuguese')

# Criando uma função para limpar o texto
def limpar_texto(texto):
    # 1. Remove pontuação
    texto_sem_pontuacao = ''.join([char for char in texto if char not in string.punctuation])
    
    # 2. Converte para minúsculas e divide em palavras
    palavras = texto_sem_pontuacao.lower().split()
    
    # 3. Remove stopwords
    palavras_sem_stopwords = [palavra for palavra in palavras if palavra not in stopwords]
    
    # 4. Junta as palavras de volta em uma string
    return ' '.join(palavras_sem_stopwords)

# Vendo um exemplo antes e depois
print("TEXTO ORIGINAL:")
print(df_final['texto'].iloc[1]) # Pega o segundo texto do nosso dataframe

print("\nTEXTO LIMPO:")
print(limpar_texto(df_final['texto'].iloc[1]))

# Apricando a função de limpeza a toda a coluna 'texto'
df_final['texto_limpo'] = df_final['texto'].apply(limpar_texto)

# Exibindo o resultado final com a nova coluna
print("\nDataFrame com a coluna de texto limpo:")
print(df_final.head())

In [ ]:
# --- MELHORIA 1: USANDO N-GRAMS ---

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# 1. Definir X e y (sem alteração)
X = df_final['texto_limpo']
y = df_final['sentimento']

# 2. Dividir em Treino e Teste (sem alteração)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Vetorização com TF-IDF e N-grams
vectorizer_ngram = TfidfVectorizer(ngram_range=(1, 2))
X_train_vec = vectorizer_ngram.fit_transform(X_train)

# Transformando os dados de TESTE, como é o correto.
X_test_vec = vectorizer_ngram.transform(X_test)

# 4. Treinamento do Modelo (sem alteração)
model_ngram = MultinomialNB()
model_ngram.fit(X_train_vec, y_train)

# 5. Fazer Previsões e Avaliar o novo modelo
y_pred_ngram = model_ngram.predict(X_test_vec)

# Calcular e imprimir a acurácia
accuracy_ngram = accuracy_score(y_test, y_pred_ngram)
print(f"Acurácia do modelo com N-grams: {accuracy_ngram * 100:.2f}%")

# Imprimir o relatório de classificação detalhado
print("\nRelatório de Classificação com N-grams:")
print(classification_report(y_test, y_pred_ngram))

In [ ]:
# --- MELHORIA 2: USANDO REGRESSÃO LOGÍSTICA COM CLASS_WEIGHT ---

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression # <--- MUDANÇA: Importando o novo modelo
from sklearn.metrics import accuracy_score, classification_report

# As etapas 1 e 2 continuam as mesmas
X = df_final['texto_limpo']
y = df_final['sentimento']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# A vetorização com N-grams continua a mesma
vectorizer_ngram = TfidfVectorizer(ngram_range=(1, 2))
X_train_vec = vectorizer_ngram.fit_transform(X_train)
X_test_vec = vectorizer_ngram.transform(X_test)

# Usando max_iter=1000 para garantir que o modelo tenha tempo de convergir
# Usando class_weight='balanced' para forçar o modelo a prestar atenção nas classes minoritárias
model_lr = LogisticRegression(random_state=42, max_iter=1000, class_weight='balanced')
model_lr.fit(X_train_vec, y_train)

# 5. Fazer Previsões e Avaliar o modelo de Regressão Logística
y_pred_lr = model_lr.predict(X_test_vec)

# Calcular e imprimir a acurácia
accuracy_lr = accuracy_score(y_test, y_pred_lr)
print(f"Acurácia do modelo (Regressão Logística): {accuracy_lr * 100:.2f}%")

# Imprimir o relatório de classificação detalhado
print("\nRelatório de Classificação (Regressão Logística):")
print(classification_report(y_test, y_pred_lr))

In [ ]:
# --- MELHORIA 3: BALANCEANDO OS DADOS DE TREINO COM SMOTE ---

# Reimportando os requisitos
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE # <-- A NOVA IMPORTAÇÃO

# As etapas 1 e 2 continuam as mesmas
X = df_final['texto_limpo']
y = df_final['sentimento']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# A vetorização com N-grams continua a mesma
vectorizer_ngram = TfidfVectorizer(ngram_range=(1, 2))
X_train_vec = vectorizer_ngram.fit_transform(X_train)
X_test_vec = vectorizer_ngram.transform(X_test)

# 3. APLICANDO O SMOTE
# Aplicamos o SMOTE APENAS nos dados de treino vetorizados
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_vec, y_train)

# Verificando o balanceamento (opcional, mas muito útil)
print("Distribuição das classes ANTES do SMOTE:")
print(pd.Series(y_train).value_counts())
print("\nDistribuição das classes DEPOIS do SMOTE:")
print(pd.Series(y_train_resampled).value_counts())

# 4. Treinamento do modelo com os dados BALANCEADOS
# Agora não precisamos mais do class_weight='balanced', pois os dados já estão balanceados
model_smote = LogisticRegression(random_state=42, max_iter=1000)
model_smote.fit(X_train_resampled, y_train_resampled)

# 5. Fazer Previsões e Avaliar no conjunto de teste ORIGINAL
y_pred_smote = model_smote.predict(X_test_vec)

# Calcular e imprimir a acurácia
accuracy_smote = accuracy_score(y_test, y_pred_smote)
print(f"\nAcurácia do modelo (com SMOTE): {accuracy_smote * 100:.2f}%")

# Imprimir o relatório de classificação detalhado
print("\nRelatório de Classificação (com SMOTE):")
print(classification_report(y_test, y_pred_smote))